In [1]:
import requests
import yaml
import json
import pandas as pd

def fetch_swagger_doc(url):
    try:
        response = requests.get(url)
        response.raise_for_status()
        return yaml.safe_load(response.text)
    except requests.exceptions.RequestException as e:
        print(f'Error retrieving Swagger doc: {str(e)}')
        return None

def save_as_json(data, filename):
    with open(filename, 'w', encoding='utf-8') as file:
        json.dump(data, file, ensure_ascii=False, indent=4)

def save_as_excel(data, filename):
    df = pd.DataFrame(data)
    df.to_excel(filename, index=False)

def main():
    swagger_doc = fetch_swagger_doc('https://opendata.cwb.gov.tw/apidoc/v1')
    if not swagger_doc:
        return
    
    paths = swagger_doc.get('paths', {})
    data = []
    for path, methods in paths.items():
        for method, details in methods.items():
            summary = details.get('summary')
            description = details.get('description')
            tags = details.get('tags')
            
            data.append({
                '資料集代碼': path.split('/')[-1],
                '資料集名稱': summary,
                '資料集描述': description,
                '標籤': tags[0] if tags else None,
            })
    save_as_excel(data, 'data_list.xlsx')

if __name__ == '__main__':
    main()
